In [ ]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

In [ ]:
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'
data_path= 'C:/Users/minkyu/Desktop/open/'
train = pd.read_csv(data_path+"train.csv" )
test = pd.read_csv(data_path+"test.csv" )
sample = pd.read_csv(data_path+"sample_submission.csv")

In [ ]:
def preprocess(df):
    df.replace('-', np.nan, inplace=True)
    df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
    df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
    df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
    df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
    df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
    df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
    df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()

    return df


train = preprocess(train)
test = preprocess(test)


In [ ]:
from sentence_transformers import SentenceTransformer
import torch
# GPU 사용 가능 여부 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Embedding Vector 추출 모델을 GPU로 로드
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False).to(device)

In [ ]:
grouped = train.groupby(["공종(중분류)", "인적사고"])

res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    
    if len(plan) < 2:  # 데이터가 1개 이하라면 유사도 계산 불가
        continue  

    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    
    # 가장 평균 유사도가 높은 문장을 선택
    best_idx = similarity.mean(axis=1).argmax()
    cosine_res += similarity[best_idx].tolist()
    res[name] = plan.iloc[best_idx]

arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")


In [ ]:
res_v = {}
for k,v in res.items():
    res_v[k] = model.encode(v)

In [ ]:
sample.to_csv("baseline.csv", index=False, encoding='utf-8-sig')

In [ ]:
from tqdm import tqdm


# sample 데이터프레임에 값 할당 (진행 상태 표시)
for i in tqdm(range(len(test)), desc="Processing", ncols=100):  # tqdm을 사용하여 진행 상태 표시
    accident = test.loc[i, "인적사고"]  # 사고 유형
    category = test.loc[i, "공종(중분류)"]  # 공종(중분류) 값

    # (공종, 인적사고) 조합을 기준으로 res에서 재발방지대책을 가져옴
    key = (category, accident)
    if key in res:  # 해당 조합이 res에 존재하는 경우
        sample.loc[i, "재발방지대책 및 향후조치계획"] = res[key]  # 문장 할당
        
        # 벡터 값도 가져와서 할당
        vector = res_v[key]
        sample.iloc[i, 2:2+len(vector)] = vector  # 벡터 길이에 맞게 할당
        
        # 진행 상황을 출력
        if i % 100 == 0:  # 100번째마다 진행 상황 출력
            print(f"Processing {i}/{len(test)} - Accident: {accident}, Category: {category}")


In [ ]:
# 주어진 문장
sentence = "안전사고 재발 방지를 위한 작업자 교육 및 안전보호구 착용 철저, 안전계단 설치 경사 재조정 및 안전교육 실시."

# 문장의 임베딩 계산
embedding = model.encode(sentence)

# 결과 출력 (임베딩 벡터)
print(embedding)

In [ ]:
sample.iloc[42, 2:2+len(vector)] = embedding

In [ ]:
sample.iloc[160, 2:2+len(vector)]= embedding

In [ ]:
sample.to_csv(base_path+"/rule_base_v2.csv", index=False, encoding='utf-8-sig')